# Cell 1: Imports & Configuration

In [10]:
import os
import cv2
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import joblib
from glob import glob
from tqdm.auto import tqdm
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader
from IPython.display import FileLink

# --- CONFIGURATION ---
DATA_DIR = "/kaggle/input/treelogy/train_images"    # Make sure your folders are here: dataset/Species_Name/image.jpg
test_folder_dir = "/kaggle/input/treelogy/test_images"
IMG_SIZE = 256          # The paper scales images to 256px [cite: 87]
BATCH_SIZE = 32         # Batch size for CNN extraction
NUM_WORKERS = 2         # CPU cores for parallel preprocessing (Kaggle usually allows 2-4)

# Setup Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

Running on: cuda


# Cell 2: Preprocessing Logic (The Core Algorithm)

In [2]:
def preprocess_leaf(img_path):
    """
    Reads an image, removes background using K-Means in LAB space, 
    and removes the stem using morphological opening.
    """
    # 1. Load Image
    img = cv2.imread(img_path)
    if img is None: return None
    
    # Resize (Longer edge -> 256px) [cite: 87]
    h, w = img.shape[:2]
    scale = IMG_SIZE / max(h, w)
    img = cv2.resize(img, (int(w * scale), int(h * scale)))
    
    # 2. Background Elimination (K-Means in LAB color space) [cite: 81-86]
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    pixels = img_lab.reshape((-1, 3))
    
    # k=2 clusters (leaf vs background)
    kmeans = KMeans(n_clusters=2, n_init=3, random_state=42)
    labels = kmeans.fit_predict(pixels)
    
    # Assume the leaf is the center pixel's cluster
    center_idx = len(labels) // 2
    leaf_label = labels[center_idx]
    
    # Create Binary Mask
    mask = (labels == leaf_label).reshape(img.shape[:2]).astype(np.uint8) * 255
    
    # 3. Refine Mask (Gaussian Blur + Otsu) [cite: 91-93]
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 4. Stem Removal (Morphological Opening with Ellipse Kernel) [cite: 106-116]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    mask_final = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    # Apply mask to original image
    result = cv2.bitwise_and(img, img, mask=mask_final)
    
    # Convert BGR (OpenCV default) to RGB (PyTorch default)
    return cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

# Cell 3: CNN Model (AlexNet Feature Extractor)

In [3]:
# Load pre-trained AlexNet
base_model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
base_model.to(device)
base_model.eval() # Set to evaluation mode

class TreelogyExtractor(torch.nn.Module):
    def __init__(self, original_model):
        super().__init__()
        self.features = original_model.features
        self.avgpool = original_model.avgpool
        # Isolate up to fc6 (Index 1 in PyTorch's classifier block)
        # Paper found fc6 + SVM gave best results (90.5%) [cite: 253]
        self.classifier = torch.nn.Sequential(*list(original_model.classifier.children())[:2])

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

extractor = TreelogyExtractor(base_model).to(device)

# Standard ImageNet normalization for CNN input
cnn_preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


100%|██████████| 233M/233M [00:01<00:00, 215MB/s] 


# Cell 4: Optimized Data Loader (For Speed)

In [18]:
# --- ENHANCED DATA LOADER WITH ROTATION  ---
class AugmentedLeafDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        # We pretend the dataset is 4x larger (0, 90, 180, 270 degrees)
        return len(self.file_list) * 4
    
    def __getitem__(self, idx):
        # 1. Determine which image and which rotation
        real_idx = idx // 4           # The actual image index
        rotation_type = idx % 4       # 0=0°, 1=90°, 2=180°, 3=270°
        
        path, label = self.file_list[real_idx]
        
        # 2. Preprocess (CPU)
        clean_leaf_rgb = preprocess_leaf(path)
        if clean_leaf_rgb is None: return None
        
        # 3. Apply Rotation
        if rotation_type == 1:
            clean_leaf_rgb = cv2.rotate(clean_leaf_rgb, cv2.ROTATE_90_CLOCKWISE)
        elif rotation_type == 2:
            clean_leaf_rgb = cv2.rotate(clean_leaf_rgb, cv2.ROTATE_180)
        elif rotation_type == 3:
            clean_leaf_rgb = cv2.rotate(clean_leaf_rgb, cv2.ROTATE_90_COUNTERCLOCKWISE)
        # rotation_type 0 does nothing (original image)

        # 4. Transform for CNN
        if self.transform:
            img_tensor = self.transform(clean_leaf_rgb)
        else:
            img_tensor = clean_leaf_rgb
            
        return img_tensor, label



# Cell 5: Main Training Loop

In [ ]:
# 1. Gather all file paths
all_files = []
classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
print(f"Found {len(classes)} classes: {classes}")

for species in classes:
    species_path = os.path.join(DATA_DIR, species)
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        for img_path in glob(os.path.join(species_path, ext)):
            all_files.append((img_path, species))

print(f"Total images to process: {len(all_files)}")

# 2. Setup Loader
train_dataset = AugmentedLeafDataset(all_files, transform=cnn_preprocess)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                    num_workers=NUM_WORKERS, collate_fn=collate_fn)

print(f"Dataset virtually expanded to {len(train_dataset)} images via rotation.")

# 3. Extract Features
X = []
y = []

print("Starting Feature Extraction...")
with torch.no_grad():
    # tqdm creates the progress bar
    for images, labels in tqdm(train_loader, desc="Processing Batches"):
        if images.numel() == 0: continue
        
        # Move batch to GPU
        images = images.to(device)
        
        # Forward pass (Get fc6 vectors)
        features = extractor(images)
        
        # Store results
        X.append(features.cpu().numpy())
        y.extend(labels)

# Concatenate all batches
X = np.concatenate(X, axis=0)
y = np.array(y)
print(f"Extraction Complete. Feature Matrix: {X.shape}")

Found 57 classes: ['n01440000', 'n01448192', 'n01456384', 'n01464576', 'n01472768', 'n01480960', 'n01489152', 'n01497344', 'n01505536', 'n01513728', 'n01521920', 'n01530112', 'n01538304', 'n01546496', 'n01554688', 'n01562880', 'n01571072', 'n01579264', 'n01587456', 'n01595648', 'n01603840', 'n01612032', 'n01620224', 'n01628416', 'n01636608', 'n01644800', 'n01652992', 'n01661184', 'n01669376', 'n01677568', 'n01685760', 'n01693952', 'n01702144', 'n01710336', 'n01718528', 'n01726720', 'n01734912', 'n01743104', 'n01751296', 'n01759488', 'n01767680', 'n01775872', 'n01784064', 'n01792256', 'n01800448', 'n01808640', 'n01816832', 'n01825024', 'n01833216', 'n01841408', 'n01849600', 'n01857792', 'n01865984', 'n01874176', 'n01882368', 'n01890560', 'n01898752']
Total images to process: 18992
Dataset virtually expanded to 75968 images via rotation.
Starting Feature Extraction...


Processing Batches:   0%|          | 0/2374 [00:00<?, ?it/s]

# Cell 6: Train SVM & Save Model

In [20]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV

# 1. Use the current X and y (assuming they are still in memory)
# If you lost them, you sadly have to re-run the extraction steps.

# 2. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Use SGDClassifier (The Fast "Approximate" SVM)
# loss='hinge' makes it behave exactly like a Linear SVM
# alpha=0.0001 is the regularization (similar to C in SVM)
# n_jobs=-1 uses all CPU cores
fast_svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, n_jobs=-1, random_state=42)

# Optional: Wrap in CalibratedClassifierCV if you want probability scores later
clf = make_pipeline(StandardScaler(), fast_svm)

print("Fitting Fast SVM Classifier...")
clf.fit(X_train, y_train)

# 4. Evaluate
preds = clf.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"\n========================================")
print(f"Final Accuracy: {acc*100:.2f}%")
print(f"========================================")

# 5. Save Model & Labels (So you can sleep!)
joblib.dump(clf, 'treelogy_svm_model.pkl')
joblib.dump(classes, 'class_labels.pkl')
print("Model saved as 'treelogy_svm_model.pkl'")

display(FileLink('treelogy_svm_model.pkl'))
display(FileLink('class_labels.pkl'))

Fitting Fast SVM Classifier...

Final Accuracy: 77.52%
Model saved as 'treelogy_svm_model.pkl'


/kaggle/working/treelogy_svm_model.pkl

/kaggle/working/class_labels.pkl

# Cell 7: (Optional) Test/Inference Code

## Test loop

In [21]:
# 1. Gather all file paths
all_files = []
classes = sorted([d for d in os.listdir(test_folder_dir) if os.path.isdir(os.path.join(test_folder_dir, d))])
print(f"Found {len(classes)} classes: {classes}")

for species in classes:
    species_path = os.path.join(test_folder_dir, species)
    for ext in ("*.jpg", "*.jpeg", "*.png"):
        for img_path in glob(os.path.join(species_path, ext)):
            all_files.append((img_path, species))

print(f"Total images to process: {len(all_files)}")

# Standard Dataset (No Rotation/Augmentation)
class LeafDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        path, label = self.file_list[idx]
        
        # Preprocess (CPU)
        clean_leaf_rgb = preprocess_leaf(path)
        
        if clean_leaf_rgb is None:
            return None # Handle bad images
            
        # Transform for CNN
        if self.transform:
            img_tensor = self.transform(clean_leaf_rgb)
        else:
            img_tensor = clean_leaf_rgb
            
        return img_tensor, label

# Helper to skip failed images
def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return torch.tensor([]), [] 
    return torch.utils.data.dataloader.default_collate(batch)
    

# 2. Setup Loader
test_dataset = LeafDataset(all_files, transform=cnn_preprocess)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                    num_workers=NUM_WORKERS, collate_fn=collate_fn)

# 3. Extract Features
X_test = []
y_test = []

print("Starting Feature Extraction...")
with torch.no_grad():
    # tqdm creates the progress bar
    for images, labels in tqdm(test_loader, desc="Processing Batches"):
        if images.numel() == 0: continue
        
        # Move batch to GPU
        images = images.to(device)
        
        # Forward pass (Get fc6 vectors)
        features = extractor(images)
        
        # Store results
        X_test.append(features.cpu().numpy())
        y_test.extend(labels)

# Concatenate all batches
X_test = np.concatenate(X_test, axis=0)
y_test = np.array(y_test)
print(f"Extraction Complete. Feature Matrix: {X_test.shape}")

Found 57 classes: ['t01440000', 't01448192', 't01456384', 't01464576', 't01472768', 't01480960', 't01489152', 't01497344', 't01505536', 't01513728', 't01521920', 't01530112', 't01538304', 't01546496', 't01554688', 't01562880', 't01571072', 't01579264', 't01587456', 't01595648', 't01603840', 't01612032', 't01620224', 't01628416', 't01636608', 't01644800', 't01652992', 't01661184', 't01669376', 't01677568', 't01685760', 't01693952', 't01702144', 't01710336', 't01718528', 't01726720', 't01734912', 't01743104', 't01751296', 't01759488', 't01767680', 't01775872', 't01784064', 't01792256', 't01800448', 't01808640', 't01816832', 't01825024', 't01833216', 't01841408', 't01849600', 't01857792', 't01865984', 't01874176', 't01882368', 't01890560', 't01898752']
Total images to process: 2640
Starting Feature Extraction...


Processing Batches:   0%|          | 0/83 [00:00<?, ?it/s]

Extraction Complete. Feature Matrix: (2640, 4096)


In [22]:


preds = clf.predict(X_test)
# Create a new list where we replace the starting 't' with 'n'
# This converts 't01808640' -> 'n01808640' so it matches the model's output
y_test_fixed = [label.replace('t', 'n', 1) if label.startswith('t') else label for label in y_test]
acc = accuracy_score(y_test_fixed, preds)

print(f"\n========================================")
print(f"Testing Accuracy: {acc*100:.2f}%")
print(f"========================================")


Testing Accuracy: 67.23%


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models
from tqdm.auto import tqdm

# --- CONFIGURATION ---
NUM_EPOCHS = 10     
LEARNING_RATE = 0.001
BATCH_SIZE = 32

# 1. Setup Data Loaders (Use your Augmented Dataset!)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=2, collate_fn=collate_fn)

# 2. Prepare the Model
print("Building Fine-Tuning Model...")
model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)

# Freeze features
for param in model.features.parameters():
    param.requires_grad = False

# Replace Classifier
num_features = model.classifier[6].in_features
num_classes = len(classes)
model.classifier[6] = nn.Linear(num_features, num_classes)

model = model.to(device)

# 3. Define Training Tools
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

# 4. The Fine-Tuning Loop
print(f"Starting Fine-Tuning for {NUM_EPOCHS} epochs...")

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        if images.numel() == 0: continue
        
        images = images.to(device)
        
        # Convert string labels to indices
        label_indices = torch.tensor([classes.index(l) for l in labels]).to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward + Backward
        outputs = model(images)
        loss = criterion(outputs, label_indices)
        loss.backward()
        optimizer.step()
        
        # Stats
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        
        # --- FIX IS HERE ---
        # We calculate size first, then add to total
        batch_len = label_indices.size(0)
        total += batch_len
        # -------------------
        
        correct += (predicted == label_indices).sum().item()
        
    epoch_acc = 100 * correct / total
    print(f"Epoch {epoch+1} Result: Loss={running_loss/len(train_loader):.4f}, Acc={epoch_acc:.2f}%")

print("Fine-Tuning Complete! The model is now a 'Leaf Specialist'.")

NameError: name 'train_dataset' is not defined